# Analyzing ACS 2019 and standardized decennial Census data from 2000

This script combines and analyzes data from the 5-year estimates of the American Community Survey (2013-2017) and standardized decennial Census data from the [US2010 Longitudinal Tract Data Base](https://s4.ad.brown.edu/Projects/Diversity/Researcher/Bridging.htm) to determine the following:
* Whether a tract gentrified or not according to [this methodology](http://www.governing.com/gov-data/gentrification-report-methodology.html)
* Percent-point changes for six non-overlapping race/ethnicity groups

In [1]:
import glob

import pandas as pd
import geopandas as gp
import numpy as np

#### Load data and filter it down to metro level

* Load data from the [Longitudinal Tract Data Base](https://s4.ad.brown.edu/Projects/Diversity/Researcher/Bridging.htm). (Column definitions are specified in the LTDB"s [data dictionary](https://s4.ad.brown.edu/Projects/Diversity/Researcher/LTBDDload/Dfiles/codebooks.pdf).)
* Load data from 2019 ACS


In [2]:
CSV_OPTIONS_2000 = dict(
    encoding = "ISO-8859-1",
    dtype = {
        "state": str,
        "county":str,
        "tract": str,
        "TRTID10": str,
        "MHMVAL00": float,
        "HINC00": float
    },
    na_values = [ " " ]
)

In [3]:
full_count_data = (
    pd.read_csv(
        "../data/LTDB_Std_2000_fullcount.csv",
        **CSV_OPTIONS_2000
    )
    [[
        "TRTID10",
        "state", 
        "county", 
        "tract",
        "POP00",
        "NHWHT00",
        "NHBLK00",
        "NTV00",
        "ASIAN00",
        "HAW00",
        "HISP00",
    ]]
    .assign(
        TRTID10 = lambda df: df["TRTID10"].str.zfill(11)
    )
)

full_count_data.head()

,TRTID10,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HAW00,HISP00
0,01001020100,AL,Autauga County,Census Tract 201,1920.974695,1722.977295,144.998093,28.999619,7.999895,0.0,11.999842
1,06083002402,CA,Santa Barbara County,Census Tract 24.02,6742.000000,1573.000000,126.000000,44.000000,272.000000,11.0,4700.000000
2,01001020200,AL,Autauga County,Census Tract 202,1892.000000,671.000000,1177.000000,12.000000,12.000000,0.0,16.000000
3,06083002102,CA,Santa Barbara County,Census Tract 21.02,2041.000000,1204.000000,57.000000,19.000000,60.000000,2.0,696.000000
4,01001020300,AL,Autauga County,Census Tract 203,3339.000000,2738.000000,498.000000,16.000000,27.000000,1.0,55.000000


In [4]:
full_count_data.dtypes

TRTID10     object
state       object
county      object
tract       object
POP00      float64
NHWHT00    float64
NHBLK00    float64
NTV00      float64
ASIAN00    float64
HAW00      float64
HISP00     float64
dtype: object

In [5]:
len(full_count_data)

72693

In [6]:
sample_data = pd.read_csv(
    "../data/LTDB_Std_2000_Sample.csv",
    **CSV_OPTIONS_2000,
)

sample_data = sample_data[[
    "TRTID10", 
    "state", 
    "county", 
    "tract",
    "AG25UP00",
    "HINC00", 
    "MHMVAL00", 
    "COL00"
]]

sample_data.head()

,TRTID10,state,county,tract,AG25UP00,HINC00,MHMVAL00,COL00
0,01001020100,AL,Autauga County,Census Tract 201,1226.983887,36685.00080,76600.00185,191.997467
1,01001020200,AL,Autauga County,Census Tract 202,1157.000000,30298.00000,72900.00000,170.000000
2,01001020300,AL,Autauga County,Census Tract 203,2130.000000,46731.00000,79900.00000,478.000000
3,01001020400,AL,Autauga County,Census Tract 204,3072.000000,46142.00000,89800.00000,708.000000
4,01001020500,AL,Autauga County,Census Tract 205,3785.214923,58886.26948,116593.52740,1214.091953


In [7]:
census_data_2000 = (
    pd.merge(
        full_count_data,
        sample_data,
        on = [ "state", "county", "tract", "TRTID10" ],
        how = "left",
        validate = "1:1",
    )
    .rename(columns = {
        "TRTID10": "GEOID", 
        "POP00": "total_population", # total population
        "AG25UP00": "total_population_25_over", # total population 25 and over
        "NHWHT00": "white_alone", # persons of white race, not Hispanic origin -- equivalent of 2010 "white alone"
        "NHBLK00":"black_alone", # persons of black race, not Hispanic origin -- equivalent of 2010 "black alone"
        "NTV00": "native_alone", #  persons of Native American race
        "ASIAN00": "asian_alone", # Asian and Pacific Islander race
        "HAW00": "native_hawaiian_pacific_islander",  # persons of Hawaiian race
        "HISP00":"hispanic_or_latino", # persons of Hispanic origin 
        "HINC00": "median_income", # Median household income, total
        "MHMVAL00": "median_home_value", # Median home value
        "COL00": "educational_attainment" # persons with at least a four-year college degree
    })
)

print(len(census_data_2000), len(sample_data), len(full_count_data))
census_data_2000.head()

72693 72693 72693


,GEOID,state,county,tract,total_population,white_alone,black_alone,native_alone,asian_alone,native_hawaiian_pacific_islander,hispanic_or_latino,total_population_25_over,median_income,median_home_value,educational_attainment
0,01001020100,AL,Autauga County,Census Tract 201,1920.974695,1722.977295,144.998093,28.999619,7.999895,0.0,11.999842,1226.983887,36685.0008,76600.00185,191.997467
1,06083002402,CA,Santa Barbara County,Census Tract 24.02,6742.000000,1573.000000,126.000000,44.000000,272.000000,11.0,4700.000000,3298.000000,31803.0000,127000.00000,205.000000
2,01001020200,AL,Autauga County,Census Tract 202,1892.000000,671.000000,1177.000000,12.000000,12.000000,0.0,16.000000,1157.000000,30298.0000,72900.00000,170.000000
3,06083002102,CA,Santa Barbara County,Census Tract 21.02,2041.000000,1204.000000,57.000000,19.000000,60.000000,2.0,696.000000,1378.000000,44135.0000,144600.00000,244.000000
4,01001020300,AL,Autauga County,Census Tract 203,3339.000000,2738.000000,498.000000,16.000000,27.000000,1.0,55.000000,2130.000000,46731.0000,79900.00000,478.000000


#### Join with 2019 Census data

Note: The [Census data dictionary](https://www.census.gov/data/developers/data-sets/acs-5year/data-notes.html) suggests that `-666666666` is a placeholder value for data that is not available:
```A "-" entry in the estimate column indicates that either no sample observations or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest interval or upper interval of an open-ended distribution.```

In [8]:
census_data_2019 = pd.read_csv(
    "../output/census_tracts.csv",
    dtype = { "geoid": str },
    na_values = [ -666666666 ]
).rename(
    columns = {"geoid": "GEOID" }
)

print(len(census_data_2019))
census_data_2019.head()

60341


,GEOID,name,total_population,total_population_25_over,median_income,median_home_value,educational_attainment,white_alone,black_alone,native_alone,asian_alone,native_hawaiian_pacific_islander,some_other_race_alone,two_or_more,hispanic_or_latino,metro_area
0,48059030101,"Census Tract 301.01, Callahan County, Texas",4888.0,3161.0,49736.0,115100.0,652.0,4183.0,105.0,0.0,58.0,0.0,0.0,182.0,360.0,"Abilene, TX (Metropolitan Statistical Area)"
1,48059030102,"Census Tract 301.02, Callahan County, Texas",4225.0,3310.0,53063.0,90900.0,669.0,3774.0,19.0,0.0,0.0,0.0,0.0,152.0,280.0,"Abilene, TX (Metropolitan Statistical Area)"
2,48059030200,"Census Tract 302, Callahan County, Texas",4743.0,3368.0,43430.0,82500.0,749.0,3959.0,2.0,5.0,11.0,0.0,0.0,72.0,694.0,"Abilene, TX (Metropolitan Statistical Area)"
3,48253020102,"Census Tract 201.02, Jones County, Texas",6856.0,5985.0,NaN,NaN,120.0,1733.0,2420.0,107.0,73.0,0.0,11.0,412.0,2182.0,"Abilene, TX (Metropolitan Statistical Area)"
4,48253020500,"Census Tract 205, Jones County, Texas",4276.0,2932.0,59318.0,118600.0,547.0,3801.0,30.0,16.0,0.0,0.0,5.0,52.0,372.0,"Abilene, TX (Metropolitan Statistical Area)"


In [9]:
census_data_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60341 entries, 0 to 60340
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   GEOID                             60341 non-null  object 
 1   name                              60341 non-null  object 
 2   total_population                  60341 non-null  float64
 3   total_population_25_over          60341 non-null  float64
 4   median_income                     59519 non-null  float64
 5   median_home_value                 58612 non-null  float64
 6   educational_attainment            60341 non-null  float64
 7   white_alone                       60341 non-null  float64
 8   black_alone                       60341 non-null  float64
 9   native_alone                      60341 non-null  float64
 10  asian_alone                       60341 non-null  float64
 11  native_hawaiian_pacific_islander  60341 non-null  float64
 12  some

In [10]:
census_merged = pd.merge(
    census_data_2019,
    census_data_2000, 
    on = "GEOID",
    how = "left",
    suffixes = [ "_19", "_00" ],
    validate = "1:1",
)

census_merged["median_home_value_00"] = census_merged["median_home_value_00"].replace(0, np.nan)

census_merged.head()

,GEOID,name,total_population_19,total_population_25_over_19,median_income_19,median_home_value_19,educational_attainment_19,white_alone_19,black_alone_19,native_alone_19,...,white_alone_00,black_alone_00,native_alone_00,asian_alone_00,native_hawaiian_pacific_islander_00,hispanic_or_latino_00,total_population_25_over_00,median_income_00,median_home_value_00,educational_attainment_00
0,48059030101,"Census Tract 301.01, Callahan County, Texas",4888.0,3161.0,49736.0,115100.0,652.0,4183.0,105.0,0.0,...,4192.939941,20.969942,42.988380,27.260923,2.096994,250.066559,2980.877197,34026.00115,51299.99911,344.955536
1,48059030102,"Census Tract 301.02, Callahan County, Texas",4225.0,3310.0,53063.0,90900.0,669.0,3774.0,19.0,0.0,...,3805.076190,19.040321,39.011744,24.739257,1.903006,226.942102,2705.146980,34026.00669,51299.99066,313.045779
2,48059030200,"Census Tract 302, Callahan County, Texas",4743.0,3368.0,43430.0,82500.0,749.0,3959.0,2.0,5.0,...,3823.999512,4.000000,33.999996,21.999998,0.000000,334.999970,2971.999756,27754.00052,45799.99864,406.999939
3,48253020102,"Census Tract 201.02, Jones County, Texas",6856.0,5985.0,NaN,NaN,120.0,1733.0,2420.0,107.0,...,0.000299,1.000190,0.000002,0.000003,0.000000,0.000160,0.000448,37395.99969,46899.99970,0.000024
4,48253020500,"Census Tract 205, Jones County, Texas",4276.0,2932.0,59318.0,118600.0,547.0,3801.0,30.0,16.0,...,4501.477935,30.008129,50.005826,25.002303,1.000090,365.081290,3200.376313,36291.57750,45998.84549,335.052030


In [11]:
census_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60341 entries, 0 to 60340
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   GEOID                                60341 non-null  object 
 1   name                                 60341 non-null  object 
 2   total_population_19                  60341 non-null  float64
 3   total_population_25_over_19          60341 non-null  float64
 4   median_income_19                     59519 non-null  float64
 5   median_home_value_19                 58612 non-null  float64
 6   educational_attainment_19            60341 non-null  float64
 7   white_alone_19                       60341 non-null  float64
 8   black_alone_19                       60341 non-null  float64
 9   native_alone_19                      60341 non-null  float64
 10  asian_alone_19                       60341 non-null  float64
 11  native_hawaiian_pacific_isla

In [12]:
census_merged["metro_area"].value_counts()

New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)          4542
Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)             2929
Chicago-Naperville-Elgin, IL-IN-WI (Metropolitan Statistical Area)             2215
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD (Metropolitan Statistical Area)    1477
Dallas-Fort Worth-Arlington, TX (Metropolitan Statistical Area)                1312
                                                                               ... 
Blacksburg-Christiansburg, VA (Metropolitan Statistical Area)                    14
Enid, OK (Metropolitan Statistical Area)                                         12
Walla Walla, WA (Metropolitan Statistical Area)                                  12
Charlottesville, VA (Metropolitan Statistical Area)                              10
Winchester, VA-WV (Metropolitan Statistical Area)                                 5
Name: metro_area, Length: 381, dtype: int64

In [13]:
census_merged.to_csv(
    "../output/census_data_metro.csv",
    index = False
)

# The gentrification measure
It is based on two tests detailed [here](http://www.governing.com/gov-data/gentrification-report-methodology.html).

#### Test 1: does the tract qualify for gentrification?

* The tract had a population of at least 500 residents at the beginning and end of a decade and was located within a central city

* The tract’s median household income was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.

* The tract’s median home value was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.

#### Test 2: has it gentrified?

* An increase in a tract's educational attainment, as measured by the percentage of residents age 25 and over holding bachelor’s degrees, was in the top third of all tracts within a metro area.

* A tract’s median home value increased when adjusted for inflation.

* The percentage increase in a tract’s inflation-adjusted median home value was in the top third of all tracts within a metro area.

*Calculate the metro-level variables needed for these tests on a city-wide level:*

* 40th percentile household income threshold for metro level data 
* 40th percentile median home value for metro level
* bachelor's degree — top third for metro level
* inflation-adjusted median home value — top third for all metro level

### Educational attainment percentage change

In [14]:
census_merged["educational_attainment_pct_19"] = (
    100 * census_merged["educational_attainment_19"] /
    census_merged["total_population_25_over_19"]
)

census_merged["educational_attainment_pct_00"] = (
    100 * census_merged["educational_attainment_00"] / 
    census_merged["total_population_25_over_00"]
)

census_merged["educational_attainment_change"] = (
    census_merged["educational_attainment_pct_19"] - 
    census_merged["educational_attainment_pct_00"]
)

### Home value percentage change

Per [BLS CPI inflation rate](https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1.00&year1=200001&year2=201701), 1 dollar in January 2000 was worth $1.44 in January 2017.

In [15]:
inflation_rate  = 1.49

Now calculate the varios thresholds for our gentrification tests: 
* `40th percentile household income threshold for metro level data`
* `40th percentile median home value for metro level` 
* `inflation-adjusted median home value — top third percentile for all metro level`

In [16]:
census_merged["home_pct_change"] = (
    (
        census_merged["median_home_value_19"] - # current median home value
        (census_merged["median_home_value_00"] * inflation_rate) # previous median home value adjusted for inflation
    ) / 
    census_merged["median_home_value_19"] # current median home value 
)

### Calculate percentiles 

In [17]:
census_merged_grp = census_merged.groupby("metro_area")

metro_metrics = (
    pd.DataFrame({
        "metro_median_income_00_q40": census_merged_grp["median_income_00"].quantile(0.4),
        "metro_median_home_value_00_q40": census_merged_grp["median_home_value_00"].quantile(0.4),
        "educational_attainment_change_q66": census_merged_grp["educational_attainment_change"].quantile(2.0/3),
        "home_pct_change_q66": census_merged_grp["home_pct_change"].quantile(2.0/3)
    })
    .reset_index()
)

metro_metrics

,metro_area,metro_median_income_00_q40,metro_median_home_value_00_q40,educational_attainment_change_q66,home_pct_change_q66
0,"Abilene, TX (Metropolitan Statistical Area)",27839.167988,40619.924876,5.984326,0.274358
1,"Akron, OH (Metropolitan Statistical Area)",38404.400000,96955.519310,8.176394,-0.174332
2,"Albany, GA (Metropolitan Statistical Area)",28269.000000,60200.000000,3.249389,0.035057
3,"Albany-Lebanon, OR (Metropolitan Statistical A...",33618.000000,117000.000000,7.774160,0.175818
4,"Albany-Schenectady-Troy, NY (Metropolitan Stat...",41766.081516,93863.382344,9.626630,0.251047
...,...,...,...,...,...
376,"Yakima, WA (Metropolitan Statistical Area)",31690.200000,85320.000000,2.267561,0.119924
377,"York-Hanover, PA (Metropolitan Statistical Area)",44456.921964,97420.000000,8.225153,0.113234
378,"Youngstown-Warren-Boardman, OH-PA (Metropolita...",33065.400000,68180.000000,6.221313,-0.105607
379,"Yuba City, CA (Metropolitan Statistical Area)",31777.000152,91560.000000,6.121047,0.370051


In [18]:
census_merged_with_metro = pd.merge(
    census_merged,
    metro_metrics,
    on = "metro_area",
    how = "left",
)

Merge with walkover spreadsheet:
- open walkover `cities_metroareas_tracts_walkover.csv`
- merge city walkover column with this `census_merged_with_metro`. If multiple cities are in the same metroarea, create a second row for that city


In [19]:
walkover_data = pd.read_csv("../data/cities_metroareas_tracts_walkover.csv")

print(len(walkover_data))
walkover_data.head()

50


,city,population_2019,metro_area_name,census_reporter_file,census_reporter_link
0,"New York city, New York",8336817,"New York-Newark-Jersey City, NY-NJ-PA (Metropo...",acs2019_5yr_B03002_14000US36081032000,https://censusreporter.org/data/table/?table=B...
1,"Los Angeles city, California",3979576,"Los Angeles-Long Beach-Anaheim, CA (Metropolit...",acs2019_5yr_B03002_14000US06037123800,https://censusreporter.org/data/table/?table=B...
2,"Chicago city, Illinois",2693976,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",acs2019_5yr_B03002_14000US17031081600,https://censusreporter.org/data/table/?table=B...
3,"Houston city, Texas",2320268,"Houston-The Woodlands-Sugar Land, TX (Metropol...",acs2019_5yr_B03002_14000US48201410100,https://censusreporter.org/data/table/?table=B...
4,"Phoenix city, Arizona",1680992,"Phoenix-Mesa-Chandler, AZ (Metropolitan Statis...",acs2019_5yr_B03002_14000US04013611600,https://censusreporter.org/data/table/?table=B...


In [21]:
# of the most populous cities in the US how many are in the same metroarea and where?
walkover_data["metro_area_name"].value_counts()

Dallas-Fort Worth-Arlington, TX (Metropolitan Statistical Area)                   3
Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)                2
Phoenix-Mesa-Chandler, AZ (Metropolitan Statistical Area)                         2
San Francisco-Oakland-Berkeley, CA (Metropolitan Statistical Area)                2
New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)             1
Atlanta-Sandy Springs-Alpharetta, GA (Metropolitan Statistical Area)              1
Louisville/Jefferson County, KY-IN (Metropolitan Statistical Area)                1
Baltimore-Columbia-Towson, MD (Metropolitan Statistical Area)                     1
Milwaukee-Waukesha, WI (Metropolitan Statistical Area)                            1
Albuquerque, NM (Metropolitan Statistical Area)                                   1
Tucson, AZ (Metropolitan Statistical Area)                                        1
Fresno, CA (Metropolitan Statistical Area)                                  

In [22]:
census_merged_with_metro

,GEOID,name,total_population_19,total_population_25_over_19,median_income_19,median_home_value_19,educational_attainment_19,white_alone_19,black_alone_19,native_alone_19,...,median_home_value_00,educational_attainment_00,educational_attainment_pct_19,educational_attainment_pct_00,educational_attainment_change,home_pct_change,metro_median_income_00_q40,metro_median_home_value_00_q40,educational_attainment_change_q66,home_pct_change_q66
0,48059030101,"Census Tract 301.01, Callahan County, Texas",4888.0,3161.0,49736.0,115100.0,652.0,4183.0,105.0,0.0,...,51299.99911,344.955536,20.626384,11.572283,9.054101,0.335908,27839.167988,40619.924876,5.984326,0.274358
1,48059030102,"Census Tract 301.02, Callahan County, Texas",4225.0,3310.0,53063.0,90900.0,669.0,3774.0,19.0,0.0,...,51299.99066,313.045779,20.211480,11.572228,8.639252,0.159109,27839.167988,40619.924876,5.984326,0.274358
2,48059030200,"Census Tract 302, Callahan County, Texas",4743.0,3368.0,43430.0,82500.0,749.0,3959.0,2.0,5.0,...,45799.99864,406.999939,22.238717,13.694481,8.544236,0.172824,27839.167988,40619.924876,5.984326,0.274358
3,48253020102,"Census Tract 201.02, Jones County, Texas",6856.0,5985.0,NaN,NaN,120.0,1733.0,2420.0,107.0,...,46899.99970,0.000024,2.005013,5.447815,-3.442802,NaN,27839.167988,40619.924876,5.984326,0.274358
4,48253020500,"Census Tract 205, Jones County, Texas",4276.0,2932.0,59318.0,118600.0,547.0,3801.0,30.0,16.0,...,45998.84549,335.052030,18.656207,10.469145,8.187063,0.422106,27839.167988,40619.924876,5.984326,0.274358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60336,04027000200,"Census Tract 2, Yuma County, Arizona",3419.0,2029.0,41856.0,117100.0,244.0,510.0,131.0,98.0,...,63900.00000,216.000000,12.025628,10.364683,1.660945,0.186926,31101.643884,69099.999396,5.405144,0.208408
60337,04027000302,"Census Tract 3.02, Yuma County, Arizona",4492.0,2369.0,20712.0,92100.0,165.0,217.0,0.0,189.0,...,60400.00000,112.000000,6.964964,4.997769,1.967195,0.022845,31101.643884,69099.999396,5.405144,0.208408
60338,04027001100,"Census Tract 11, Yuma County, Arizona",5722.0,3501.0,37071.0,139000.0,626.0,2064.0,13.0,63.0,...,86500.00000,460.000000,17.880606,13.876320,4.004286,0.072770,31101.643884,69099.999396,5.405144,0.208408
60339,04027011501,"Census Tract 115.01, Yuma County, Arizona",2289.0,1332.0,32420.0,143000.0,116.0,249.0,1.0,559.0,...,58500.00000,83.000000,8.708709,5.874027,2.834682,0.390455,31101.643884,69099.999396,5.405144,0.208408


### Select only tracts in the cities of interest

In [26]:
paths = glob.glob("../data/city_tracts/*")
csv_paths = []

for path in paths:
    csv_paths.append([
        path.split("../data/city_tracts/")[1], 
        path+"/"+ path.split("../data/city_tracts/")[1]+".csv"
    ])


csv_paths[0]

['acs2019_5yr_B03002_14000US47037010106',
 '../data/city_tracts/acs2019_5yr_B03002_14000US47037010106/acs2019_5yr_B03002_14000US47037010106.csv']

In [ ]:
# CITY_CSVS = {
#     "Atlanta": "../data/city_tracts/atlanta_census_reporter/acs2018_5yr_B03002_14000US13121008201.csv", 
#     "Baltimore": "../data/city_tracts/baltimore_census_reporter/acs2018_5yr_B03002_14000US24510260700.csv",
#     "New York City": "../data/city_tracts/new_york_census_reporter/acs2018_5yr_B03002_14000US36047990100.csv",
#     "Oakland": "../data/city_tracts/oakland_census_reporter/acs2018_5yr_B03002_14000US06001403400.csv",
#     "Washington":"../data/city_tracts/washington_census_reporter/acs2018_5yr_B03002_14000US11001007403.csv"
# }

In [40]:
tracts_in_cities = (
    pd.concat([
        pd.read_csv(
            path, usecols = [ "geoid" ]
        ).assign(
            census_reporter_file = file
        )
        for file, path in csv_paths
    ])
    .assign(
        GEOID = lambda df: df["geoid"].str.slice(7, None)
    )
    
    .drop(columns = [ "geoid" ])
)

print(len(tracts_in_cities))

tracts_in_cities.head()

13116


,census_reporter_file,GEOID
0,acs2019_5yr_B03002_14000US47037010106,47037010103
1,acs2019_5yr_B03002_14000US47037010106,47037010104
2,acs2019_5yr_B03002_14000US47037010106,47037010105
3,acs2019_5yr_B03002_14000US47037010106,47037010106
4,acs2019_5yr_B03002_14000US47037010106,47037010201


In [47]:
len(tracts_in_cities["census_reporter_file"].notna())

13116

In [50]:
tracts_in_cities_walkover = pd.merge( 
    tracts_in_cities,
    walkover_data[["census_reporter_file", "city"]], 
    on ="census_reporter_file",
    how = "left"
    
    
)

print(len(tracts_in_cities_walkover))
tracts_in_cities_walkover.head()

13116


,census_reporter_file,GEOID,city
0,acs2019_5yr_B03002_14000US47037010106,47037010103,Nashville-Davidson metropolitan government (ba...
1,acs2019_5yr_B03002_14000US47037010106,47037010104,Nashville-Davidson metropolitan government (ba...
2,acs2019_5yr_B03002_14000US47037010106,47037010105,Nashville-Davidson metropolitan government (ba...
3,acs2019_5yr_B03002_14000US47037010106,47037010106,Nashville-Davidson metropolitan government (ba...
4,acs2019_5yr_B03002_14000US47037010106,47037010201,Nashville-Davidson metropolitan government (ba...


In [53]:
cities_only = pd.merge(
    tracts_in_cities_walkover,
    census_merged_with_metro,
    on = "GEOID",
    how = "inner", #<-- specify inner instead of left
#     validate = "1:1"
)

print(len(cities_only))

cities_only.head()

13115


,census_reporter_file,GEOID,city,name,total_population_19,total_population_25_over_19,median_income_19,median_home_value_19,educational_attainment_19,white_alone_19,...,median_home_value_00,educational_attainment_00,educational_attainment_pct_19,educational_attainment_pct_00,educational_attainment_change,home_pct_change,metro_median_income_00_q40,metro_median_home_value_00_q40,educational_attainment_change_q66,home_pct_change_q66
0,acs2019_5yr_B03002_14000US47037010106,47037010103,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.03, Davidson County, Tennessee",2533.0,1872.0,70341.0,228000.0,477.0,2441.0,...,129386.5630,255.742443,25.480769,15.198157,10.282612,0.154447,41888.41521,102400.00024,12.203071,0.284405
1,acs2019_5yr_B03002_14000US47037010106,47037010104,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.04, Davidson County, Tennessee",2955.0,2349.0,64844.0,199600.0,576.0,2713.0,...,129399.9974,346.963013,24.521073,15.200000,9.321073,0.034038,41888.41521,102400.00024,12.203071,0.284405
2,acs2019_5yr_B03002_14000US47037010106,47037010105,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.05, Davidson County, Tennessee",4435.0,3621.0,64688.0,206200.0,1261.0,949.0,...,88900.0007,697.510376,34.824634,24.306243,10.518391,0.357609,41888.41521,102400.00024,12.203071,0.284405
3,acs2019_5yr_B03002_14000US47037010106,47037010106,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.06, Davidson County, Tennessee",3397.0,2301.0,63864.0,209100.0,609.0,1009.0,...,88897.6267,283.008564,26.466754,24.303559,2.163195,0.366535,41888.41521,102400.00024,12.203071,0.284405
4,acs2019_5yr_B03002_14000US47037010106,47037010201,Nashville-Davidson metropolitan government (ba...,"Census Tract 102.01, Davidson County, Tennessee",4157.0,3163.0,62829.0,214900.0,1016.0,1870.0,...,133753.4506,477.955636,32.121404,19.009903,13.111500,0.072626,41888.41521,102400.00024,12.203071,0.284405


### Run gentrification tests described above

Create columns for every test that needs to be passed:

1. First whether they qualify
2. Whether they gentrified

In [56]:
# The tract had a population of at least 500 residents at the beginning and 
# end of a decade and was located within a central city
cities_only["low_population"] = (
    (cities_only["total_population_00"] < 500) | 
    (cities_only["total_population_19"] < 500)
)  

# The tract’s median household income was in the bottom 40th percentile when compared 
# to all tracts within its metro area at the beginning of the decade.
cities_only["median_home_value_00_qualifies"] = (
    cities_only["median_home_value_00"] < cities_only["metro_median_home_value_00_q40"]
)

# The tract’s median home value was in the bottom 40th percentile when compared 
# to all tracts within its metro area at the beginning of the decade.
cities_only["median_40th_income_00_qualifies"] = (
    cities_only["median_income_00"] < cities_only["metro_median_income_00_q40"]
)

In [57]:
# An increase in a tract"s educational attainment, as measured by the 
# percentage of residents age 25 and over who hold bachelor’s degrees, 
# was in the top third percentile of all tracts within a metro area.
cities_only["educational_increase_is_top_3rd"] = (
    cities_only["educational_attainment_change"] > cities_only["educational_attainment_change_q66"]
)

# The median home value increased, after accounting for inflation
cities_only["median_home_increased"] = (
    cities_only["median_home_value_19"] > (cities_only["median_home_value_00"] * inflation_rate)
)

# The percentage increase in a tract’s inflation-adjusted median home 
# value was in the top third percentile of all tracts within a metro area.
cities_only["median_home_increase_is_top_3rd"] = (
    cities_only["home_pct_change"] > cities_only["home_pct_change_q66"]
)

In [58]:
cities_only["eligible_for_gentrification"] = (
  (cities_only["low_population"] == False) &
  (cities_only["median_home_value_00_qualifies"] == True) &
  (cities_only["median_40th_income_00_qualifies"] == True)
)

cities_only["eligible_for_gentrification"].sum()

4982

In [59]:
cities_only["gentrified"] = (
    (cities_only["eligible_for_gentrification"] == True) & 
    (cities_only["educational_increase_is_top_3rd"] == True) & 
    (cities_only["median_home_increased"] == True) & 
    (cities_only["median_home_increase_is_top_3rd"] == True)
)

cities_only["gentrified"].sum()

1117

In [60]:
cities_only["eligible_not_gentrified_highpop"] = (
  (cities_only["eligible_for_gentrification"] == True) &
  (cities_only["gentrified"] == False)
)

cities_only["eligible_not_gentrified_highpop"].sum()

3865

In [61]:
cities_only["not_eligible_highpop"] = (
  (cities_only["low_population"] == False) &
  (cities_only["eligible_for_gentrification"] == False) 
)

cities_only["not_eligible_highpop"].sum()

7766

In [62]:
(
    cities_only
    [[
        "city",
        "eligible_for_gentrification",
        "gentrified",
        "not_eligible_highpop",
        "low_population",
    ]]
    .groupby("city")
    .sum()
    .astype(int)
)

,eligible_for_gentrification,gentrified,not_eligible_highpop,low_population
city,,,,
"Albuquerque city, New Mexico",41,9,103,1
"Arlington city, Texas",25,1,57,1
"Atlanta city, Georgia",74,32,64,3
"Austin city, Texas",71,34,143,7
"Baltimore city, Maryland",169,38,30,1
"Boston city, Massachusetts",61,18,106,14
"Charlotte city, North Carolina",43,11,154,9
"Chicago city, Illinois",325,51,475,5
"Colorado Springs city, Colorado",37,10,70,4


## Calculate race/ethnicity percentage changes

In [66]:
RACE_ETHNICITY_GROUPS = [
    "white_alone",
    "black_alone",
    "asian_alone",
    "native_alone",
    "native_hawaiian_pacific_islander",
    "hispanic_or_latino",
]

for group in RACE_ETHNICITY_GROUPS:
    cities_only[f"pct_{group}_19"] = (100 * cities_only[f"{group}_19"] / cities_only["total_population_19"]).round(3)
    cities_only[f"pct_{group}_00"] = (100 * cities_only[f"{group}_00"] / cities_only["total_population_00"]).round(3)
    
    cities_only[f"pct_{group}_change"] = (cities_only[f"pct_{group}_19"] - cities_only[f"pct_{group}_00"]).round(3)

In [67]:
cities_only.to_csv(
    "../output/gentrification.csv",
    index = False
)

## Merge data with shapefiles

Shapefiles sourced here: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2019&layergroup=Census+Tracts

In [78]:
states_paths = glob.glob("../data/censusTracts/states/*")
states_shape_paths = []
for state_path in states_paths:
    states_shape_paths.append(
        state_path + "/"+ state_path.split("../data/censusTracts/states/")[1] + ".shp"
    )

states_shape_paths[0]

'../data/censusTracts/states/tl_2019_10_tract/tl_2019_10_tract.shp'

In [79]:

# {
#     "DC":"../data/censusTracts/states/tl_2019_11_DC_tract/tl_2019_11_tract.shp",
#     "Georgia": "../data/censusTracts/states/tl_2019_13_georgia_tract/tl_2019_13_tract.shp", 
#     "Maryland": "../data/censusTracts/states/tl_2019_24_maryland_tract/tl_2019_24_tract.shp",
#     "California": "../data/censusTracts/states/tl_2019_06_california_tract/tl_2019_06_tract.shp",
#     "New York": "../data/censusTracts/states/tl_2019_36_NY_tract/tl_2019_36_tract.shp",
# }


shape_files = (
        pd.concat([
            gp.read_file(path)
            .drop(columns = [
                "STATEFP",
                "COUNTYFP",
                "TRACTCE",
                "NAMELSAD",
                "NAME",
                "FUNCSTAT",
                "ALAND",
                "AWATER",
                "MTFCC"

            ])
        for path in states_shape_paths
    ])
)

shape_files.head()

,GEOID,INTPTLAT,INTPTLON,geometry
0,10003010800,+39.7777142,-075.5264969,"POLYGON ((-75.54492 39.77764, -75.54483 39.777..."
1,10005051007,+38.6944980,-075.1130691,"POLYGON ((-75.13420 38.70986, -75.13420 38.709..."
2,10005050408,+38.6344590,-075.5268421,"POLYGON ((-75.59333 38.64054, -75.59303 38.640..."
3,10003014909,+39.6599172,-075.6228934,"POLYGON ((-75.64209 39.66264, -75.64202 39.662..."
4,10005050601,+38.6014626,-075.3754891,"POLYGON ((-75.48444 38.62040, -75.48235 38.621..."


In [80]:
type(shape_files)

geopandas.geodataframe.GeoDataFrame

In [81]:
len(shape_files)

74133

In [83]:
merged_shape_files = gp.GeoDataFrame(
    pd.merge(
        cities_only,
        shape_files, 
        on = "GEOID",
        how = "left",
#         validate = "1:1", 
    )
    .dropna(subset = [ "city" ])
    .replace([np.inf, -np.inf], np.nan)
)

print(len(cities_only), len(merged_shape_files))

13115 13115


In [84]:
type(merged_shape_files)

geopandas.geodataframe.GeoDataFrame

In [85]:
merged_shape_files.head()

,census_reporter_file,GEOID,city,name,total_population_19,total_population_25_over_19,median_income_19,median_home_value_19,educational_attainment_19,white_alone_19,...,pct_native_alone_change,pct_native_hawaiian_pacific_islander_19,pct_native_hawaiian_pacific_islander_00,pct_native_hawaiian_pacific_islander_change,pct_hispanic_or_latino_19,pct_hispanic_or_latino_00,pct_hispanic_or_latino_change,INTPTLAT,INTPTLON,geometry
0,acs2019_5yr_B03002_14000US47037010106,47037010103,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.03, Davidson County, Tennessee",2533.0,1872.0,70341.0,228000.0,477.0,2441.0,...,-0.417,0.000,0.0,0.000,0.632,0.750,-0.118,+36.3444054,-086.8608396,"POLYGON ((-86.91752 36.33976, -86.91747 36.339..."
1,acs2019_5yr_B03002_14000US47037010106,47037010104,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.04, Davidson County, Tennessee",2955.0,2349.0,64844.0,199600.0,576.0,2713.0,...,-0.192,0.102,0.0,0.102,0.000,0.750,-0.750,+36.2918321,-086.8763469,"POLYGON ((-86.97440 36.24991, -86.97244 36.252..."
2,acs2019_5yr_B03002_14000US47037010106,47037010105,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.05, Davidson County, Tennessee",4435.0,3621.0,64688.0,206200.0,1261.0,949.0,...,-0.362,0.000,0.0,0.000,2.525,0.740,1.785,+36.2504208,-086.8521501,"POLYGON ((-86.89144 36.26205, -86.89100 36.262..."
3,acs2019_5yr_B03002_14000US47037010106,47037010106,Nashville-Davidson metropolitan government (ba...,"Census Tract 101.06, Davidson County, Tennessee",3397.0,2301.0,63864.0,209100.0,609.0,1009.0,...,-0.362,0.000,0.0,0.000,4.327,0.740,3.587,+36.2610013,-086.8023491,"POLYGON ((-86.83089 36.26570, -86.83016 36.265..."
4,acs2019_5yr_B03002_14000US47037010106,47037010201,Nashville-Davidson metropolitan government (ba...,"Census Tract 102.01, Davidson County, Tennessee",4157.0,3163.0,62829.0,214900.0,1016.0,1870.0,...,0.053,0.000,0.0,0.000,11.691,0.774,10.917,+36.2882537,-086.7728157,"POLYGON ((-86.81736 36.27398, -86.81733 36.274..."


### Prepare data for maps

- drop columns that don"t need to be visualized
- make a geojson file for each city


In [86]:
merged_shape_files.columns

Index(['census_reporter_file', 'GEOID', 'city', 'name', 'total_population_19',
       'total_population_25_over_19', 'median_income_19',
       'median_home_value_19', 'educational_attainment_19', 'white_alone_19',
       'black_alone_19', 'native_alone_19', 'asian_alone_19',
       'native_hawaiian_pacific_islander_19', 'some_other_race_alone',
       'two_or_more', 'hispanic_or_latino_19', 'metro_area', 'state', 'county',
       'tract', 'total_population_00', 'white_alone_00', 'black_alone_00',
       'native_alone_00', 'asian_alone_00',
       'native_hawaiian_pacific_islander_00', 'hispanic_or_latino_00',
       'total_population_25_over_00', 'median_income_00',
       'median_home_value_00', 'educational_attainment_00',
       'educational_attainment_pct_19', 'educational_attainment_pct_00',
       'educational_attainment_change', 'home_pct_change',
       'metro_median_income_00_q40', 'metro_median_home_value_00_q40',
       'educational_attainment_change_q66', 'home_pct_change_

In [88]:
columns_for_viz = [
    "GEOID",
    "name",
    "total_population_19",
    "white_alone_19",
    "black_alone_19",
    "native_alone_19",
    "asian_alone_19",
    "native_hawaiian_pacific_islander_19",
    "hispanic_or_latino_19",
    "city",
    "gentrified",
    "pct_white_alone_change",
    "pct_black_alone_change",
    "pct_native_alone_change",
    "pct_asian_alone_change",
    "pct_hispanic_or_latino_change",
    "pct_native_hawaiian_pacific_islander_change",
    "INTPTLAT", "INTPTLON", "geometry"
]

merged_shape_files_for_viz = merged_shape_files[columns_for_viz]

In [93]:
city_list = merged_shape_files_for_viz["city"].unique().tolist()

for city in city_list:
    print(city)
    output_path = "../output/for_maps/" + str(city).lower().strip().replace("/","") + "_data.csv"
    output_path_geojson = "../output/for_maps/" + str(city).lower().strip().replace("/","") + "_data.geojson"
    
    df = merged_shape_files_for_viz[merged_shape_files_for_viz["city"] == city]
    df.drop(columns = [ "geometry" ]).to_csv(output_path, index = False)
    df.to_file(
        output_path_geojson,
        driver = "GeoJSON"
    )

Nashville-Davidson metropolitan government (balance), Tennessee
Seattle city, Washington
Raleigh city, North Carolina
San Diego city, California
Detroit city, Michigan
Tampa city, Florida
Austin city, Texas
Arlington city, Texas
Fort Worth city, Texas
Portland city, Oregon
Oakland city, California
Louisville/Jefferson County metro government (balance), Kentucky
Kansas City city, Missouri
Atlanta city, Georgia
Los Angeles city, California
Colorado Springs city, Colorado
Fresno city, California
Phoenix city, Arizona
Denver city, Colorado
Indianapolis city (balance), Indiana
Omaha city, Nebraska
San Jose city, California
Tucson city, Arizona
San Francisco city, California
Columbus city, Ohio
Dallas city, Texas
Chicago city, Illinois
Washington city, District of Columbia
San Antonio city, Texas
Minneapolis city, Minnesota
Memphis city, Tennessee
Philadelphia city, Pennsylvania
El Paso city, Texas
Miami city, Florida
New Orleans city, Louisiana
Tulsa city, Oklahoma
New York city, New York
L

---

---

---